In [1]:
from xml.dom import minidom
from typing import List

# Parser to convert to symbols and formulas

# P,Q,R
class Symbol:
    def __init__(self, value: str):
        self.value = value
    
    def __str__(self):
        return self.value

# P => Q, A => B
class Formula:
    """
    Represents a logical formula, which can be a combination of two formulas
    by a single connective (e.g., A ^ B, A ∨ B, ¬A).
    For unary operations like NOT, the second argument can be None.
    """
    def __init__(self, first, second=None, branch=None):
        # For unary operations, `branch` can indicate the operation type (e.g., NOT)
        self.branch = branch  # True for OR, False for AND, None for unary operations like NOT
        self.first = first
        self.second = second

    def __str__(self):
        if self.second is None:
            if self.branch is None:  # Unary operation NOT
                return f"¬{self.first}"
            else:
                raise ValueError("Invalid configuration for unary operation.")
        else:
            connector = "∨" if self.branch else "^"
            return f"({self.first} {connector} {self.second})"

def print_element(elem, indent=0):
    """
    Recursively prints the name and attributes of an XML element and its children,
    indented according to their depth in the hierarchy.
    
    Args:
    - elem: The current XML element to print.
    - indent: The current indentation level (depth in the tree).
    """
    # Print the current element's tag name and attributes
    
    if elem.attributes:
        for attrName, attrValue in elem.attributes.items():
            print(f" {attrName}='{attrValue}'", end='')
    print()
    
    if elem.firstChild and elem.firstChild.nodeType == elem.TEXT_NODE and elem.tagName == "variable":
        print('  ' * (indent+1) + f"{elem.firstChild.data}", end='')
        return
    else:
        print('  ' * indent + elem.tagName, end='')
        if elem.tagName != "block":
            pass

    
    # Recursively print each child element
    for child in elem.childNodes:
        if child.nodeType == child.ELEMENT_NODE:
            print_element(child, indent + 1)

def construct_formula(elem):
    """
    Constructs a Formula or Symbol object from an XML element, recursively.
    
    Args:
    - elem: The current XML element to start construction from.
    
    Returns:
    A Formula or Symbol object representing the XML structure.
    """
    # Base case: variable
    if elem.tagName == "variable":
        return Symbol(elem.firstChild.data)
    
    # Unary operation: NOT
    if elem.tagName == "not":
        child = next(child for child in elem.childNodes if child.nodeType == child.ELEMENT_NODE)
        return Formula(construct_formula(child), branch=None)
    
    # Binary operations: AND, OR, and others
    child_elements = [child for child in elem.childNodes if child.nodeType == child.ELEMENT_NODE]
    if len(child_elements) == 1:  # Handling unary operation NOT
        return Formula(construct_formula(child_elements[0]), branch=None)
    elif len(child_elements) == 2:
        first = construct_formula(child_elements[0])
        second = construct_formula(child_elements[1])
        if elem.tagName == "implies":
            is_branch = True # treat as AND
            return Formula(Formula(first, branch=None), second, branch=is_branch)
        else:
            is_branch = elem.tagName == "or"
            return Formula(first, second, branch=is_branch)
    else:
        raise ValueError("Unexpected number of child elements in formula construction")

# def main():
#     dom = minidom.parse("KRR-2024/work/output/PL-01.xml")
#     elements = dom.getElementsByTagName('block')
#     # print_element(elements[0])
    
#     kb = []
#     for elem in elements:
#         for clause in elem.childNodes:
#             if clause.nodeType == clause.ELEMENT_NODE:
#                 # print(construct_formula(clause))
#                 kb.append(construct_formula(clause))
                
#     for clause in kb:
#         print(clause)
    
#     return 
#     for elem in elements:
#         print_element(elem)
        
# if __name__ == "__main__":
#     main()

In [2]:
class Node:
    def __init__(self, _type=1):
        # 1 -> terminal (variable / symbol)
        # 2 -> non-terminal (formula)
        self._type = _type
        self.value: Formula | Symbol = None
        self.expanded = False
        self.terminated = False # for closing branch with negation
        self.children = []
    
    def __str__(self) -> str:
        return self.value.__str__()
    
# KB -> root, expand
class Tree:
    def __init__(self):
        self.root = None
        self.leaf_nodes = [] # to expand tree for tableu (OBSOLETE?)
        self.open_nodes = [] # nodes that can be expanded
        self.termination_nodes: List[Node] = [] # maintain a set of negation symbols, set for O(1) lookup
        self.open_symbol_nodes: List[Node] = []
    
    def expand(self, node: Node):
        branches = self.get_open_branches(node)
        formula: Formula | Symbol = node.value
        
        # Check for terminal condition
        # handle open symbol node case
        if isinstance(formula, Symbol):
            # self.open_symbol_nodes.append(formula)
            self.open_symbol_nodes.append(node)
            return
        # handle not operator case
        elif formula.branch is None:
            # ASSUMPTION: Demorgan already applied before 
            # execution reaching here
            # self.termination_nodes.append(formula.first)
            self.termination_nodes.append(node)
            return
        
        n1_value = formula.first
        n2_value = formula.second
        n1, n2 = Node(), Node()
        n1.value, n2.value = n1_value, n2_value
        self.open_nodes.append(n1)
        self.open_nodes.append(n2)
        
        for n in branches:
            if formula.branch:
                n.children.append(n1)
                n.children.append(n2)
            else:
                # ensures children are not branched any further
                n1.children.append(n2)
                n.children.append(n1)
        
    def close_nodes(self):
        new_symbol_nodes = []
        for i in self.open_symbol_nodes:
            for j in self.termination_nodes:
                # print(i.value.value)
                # print(j.value.first.value)
                if i.value.value == j.value.first.value:
                    # terminate and remove from open
                    i.terminated = True
                    j.terminated = True
                    pass
                else:
                    new_symbol_nodes.append(i)
        
        self.open_symbol_nodes = new_symbol_nodes.copy()
    
    def get_open_branches(self, node: Node) -> List[Node]:
        cur_nodes = [node]
        branches = []
        while len(cur_nodes) > 0:
            n1 = cur_nodes.pop()
            if len(n1.children) == 0 and not n1.terminated:
                branches.append(n1)
                continue
            for child in n1.children:
                cur_nodes.append(child)
                
        return branches
    
    def traverse(self):
        self.traverse_(self.root)
    
    def traverse_(self, node: Node):
        print(node.value)
        if len(node.children) == 0:
            return
        for child in node.children:
            self.traverse_(child)

In [3]:
dom = minidom.parse("KRR-2024/work/output/PL-01.xml")
elements = dom.getElementsByTagName('block')
# print_element(elements[0])

kb = []
for elem in elements:
    for clause in elem.childNodes:
        if clause.nodeType == clause.ELEMENT_NODE:
            # print(construct_formula(clause))
            kb.append(construct_formula(clause))
            
for clause in kb:
    print(clause)

(P ^ Q)
(¬P ∨ R)
((¬R ∨ ¬S) ∨ T)
(¬Q ∨ S)
¬T


In [4]:
tree = Tree()
tree.open_nodes = kb.copy()

In [5]:
solved = False
while len(tree.open_nodes) > 0:
    formula = tree.open_nodes.pop(0)
    print(formula)
    if not isinstance(formula, Node):
        cur_node = Node()
        cur_node.value = formula
    else:
        cur_node = formula
    # print(formula.branch)
    if tree.root is None:
        tree.root = cur_node
        
    tree.expand(cur_node)
    
    tree.close_nodes()
    
    if len(tree.get_open_branches(tree.root)) == 0:
        print("No Model Exists! Tautology proved.")
        solved = True
        break
    
if not solved:
    print("Model exists! Can't prove tautology.")

(P ^ Q)
(¬P ∨ R)
((¬R ∨ ¬S) ∨ T)
(¬Q ∨ S)
¬T
P
Q
¬P
R
(¬R ∨ ¬S)
T
¬Q
No Model Exists! Tautology proved.


In [6]:
tree.traverse()

(P ^ Q)
P
Q


In [7]:
n1 = Node()
n1.value = Formula("P","Q")

n2 = Node()
n2.value = Formula("A","B",branch=True)

n3 = Node()
n3.value = Formula("P")

n4 = Node()
n4.value = Formula("A","Q",branch=True)

n3.children = [n4]

n1.children = [n2,n3]

In [8]:
tree = Tree()
tree.root = n1

In [9]:
tree.traverse()

(P ^ Q)
(A ∨ B)
¬P
(A ∨ Q)


In [10]:
for i in tree.get_open_branches(n1):
    print(i)

(A ∨ Q)
(A ∨ B)
